In [17]:
pip install fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 2.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [26]:
import requests      # Для запросов по API
import json          # Для обработки полученных результатов
import time          # Для задержки между запросами
import os            # Для работы с файлами
import pandas as pd  # Для формирования датафрейма с результатами
import datetime
from bs4 import BeautifulSoup
import numpy as np

In [5]:
vacancy_dict_all = {}

In [8]:
for q in range(18902, 60000):
    time.sleep(0.1)
    vacancy_id=[]
    vacancy_title=[]
    url = 'https://api.hh.ru/vacancies'
    #params = {'employer_id': '2391889'}
    params = {'employer_id': q}
    print(f'\r{q} ' , end='')
    try:
        response = requests.get(url, params=params)
        vacancies = response.json()
        for vacancy in vacancies['items']:
            #print(vacancy['name'], vacancy['salary'], vacancy['area']['name'])
            vacancy_id.append(vacancy.get("id"))
            vacancy_title.append(vacancy.get("name"))
            #https://hh.ru/employer/2391889?hhtmFrom=vacancy
        vacancy_dict = dict(zip(vacancy_id, vacancy_title))
        if len(vacancy_dict) > 0:
            vacancy_dict_all.update(vacancy_dict)
    except:
        continue

59999 

In [9]:
import pickle


with open("/home/marina/hh_ru/hhru_vacancy_dict_all_60000.pickle", "wb") as f:
    pickle.dump(vacancy_dict_all, f)

In [62]:
len(vacancy_dict_all)

2841

In [68]:
# dff = pd.read_parquet('/home/marina/hh_ru/df.parquet.gzip')

In [69]:
dff

,index,Вакансия,Обязанности,Требования,Условия
0,20,Начальник отдела капитального строительства,организация работы по текущему ремонту жилых ...,опыт работы по направлению не менее 5-ти лет;\...,оформление в соответствии с ТК РФ;\nстабильная...
1,28,Главный специалист-эксперт отдела развития сфе...,рассмотрение нормативных правовых актов федер...,"высшее образование по направлению ""Экономика/...",оформление в соответствии с ТК РФ;\nстабильная...
2,29,Советник Департамента стратегического планиров...,методологическое обеспечение вопросов достиже...,"высшее образование по направлению ""Государстве...",оформление в соответствии с ТК РФ;\nстабильная...
3,30,Заместитель начальника отдела инновационных пр...,организация проведения заседаний Комитета по ...,"высшее образование по направлению ""Экономика/Ф...",оформление в соответствии с ТК РФ;\nстабильная...
4,32,Заместитель начальника сводно-аналитического о...,разработка и мониторинг программ социально-эк...,"высшее образование по направлению ""Экономика/Г...",оформление в соответствии с ТК РФ;\nстабильная...
...,...,...,...,...,...
10351,17337,Помощник печатника офсетного производства,Работа помощником печатника на печатных машин...,Опыт работы приветствуется\nОпыт работы привет...,Работа на полиграфическом предприятии ОАО Упа...
10352,17338,Печатник офсетной печати,Выполнение обязанностей печатника офсетной пе...,Опыт работы в типографии для офсетной печати ...,Работа на полиграфическом предприятии ОАО Упа...
10353,17341,Инженер-конструктор (производство медицинской ...,"Разработка эскизных, технических и рабочих про...","Опыт работы на производстве (металлообработка,...",График работы 5/2;\nСтабильная выплата заработ...
10354,17342,Инженер-конструктор (производство медицинской ...,"Разработка эскизных, технических и рабочих про...","Опыт работы на производстве (металлообработка,...",График работы 5/2;\nСтабильная выплата заработ...


In [77]:
dff =pd.DataFrame(columns=['Вакансия', 'Обязанности', 'Требования', 'Условия'])

In [78]:
def parse_to_dataframe(vacancy_dict, dff):
    q = 0
    for key, value in vacancy_dict.items():
        try:
            q +=1
            url = f'https://api.hh.ru/vacancies/{key}' # Replace with the ID of the vacancy you want to get the description for
            response = requests.get(url)
            print(f'\r{q} / {len(vacancy_dict)} ' , end='')
            vacancy = response.json()
            description = vacancy['description']
            soup = BeautifulSoup(description, 'html.parser')
            sections = []
            current_section = ''
            for tag in soup.find_all():
                if tag.name == 'strong':
                    sections.append(current_section.strip())
                    current_section = tag.text + ' '
                else:
                    current_section += str(tag)
    
            sections.append(current_section.strip())
        except KeyError:
            print(f'\rups... {q}' , end='')
            continue     
        try:
            soup = BeautifulSoup(sections[1], 'html.parser')
            li_elements_o = soup.find_all('li')
    
            li_content_o = ''
            for li in li_elements_o:
                li_content_o += li.text + '\n'
    
            # print(li_content)
    
            soup = BeautifulSoup(sections[2], 'html.parser')
            li_elements_t = soup.find_all('li')
    
            li_content_t = ''
            for li in li_elements_t:
                li_content_t += li.text + '\n'
    
            # print(li_content)
    
            soup = BeautifulSoup(sections[3], 'html.parser')
            li_elements_u = soup.find_all('li')
    
            li_content_u = ''
            for li in li_elements_u:
                li_content_u += li.text + '\n'
            new_row = {'Вакансия': value, 'Обязанности': li_content_o, 'Требования': li_content_t, 'Условия': li_content_u}
            # print(new_row)
            new_row_df = pd.DataFrame.from_dict(new_row, orient='index').T
            dff = pd.concat([dff, new_row_df], ignore_index=True)
        except:
            continue
    return dff

In [79]:
i=0
for key, value in vacancy_dict_all.items():
    try:
        time.sleep(0.1)
        i+=1
        print(f'\r{i} ' , end='')
        url = f'https://api.hh.ru/vacancies/{key}' # Replace with the ID of the vacancy you want to get the description for
        response = requests.get(url)
        vacancy = response.json()
        description = vacancy['description']
        soup = BeautifulSoup(description, 'html.parser')
        sections = []
        current_section = ''
        for tag in soup.find_all():
            if tag.name == 'strong':
                sections.append(current_section.strip())
                current_section = tag.text + ' '
            else:
                current_section += str(tag)

        sections.append(current_section.strip())
    except KeyError:
        continue
    try:
        soup = BeautifulSoup(sections[1], 'html.parser')
        li_elements_o = soup.find_all('li')

        li_content_o = ''
        for li in li_elements_o:
            li_content_o += li.text + '\n'

        soup = BeautifulSoup(sections[2], 'html.parser')
        li_elements_t = soup.find_all('li')

        li_content_t = ''
        for li in li_elements_t:
            li_content_t += li.text + '\n'

        soup = BeautifulSoup(sections[3], 'html.parser')
        li_elements_u = soup.find_all('li')

        li_content_u = ''
        for li in li_elements_u:
            li_content_u += li.text + '\n'
        new_row = {'Вакансия': value, 'Обязанности': li_content_o, 'Требования': li_content_t, 'Условия': li_content_u}
        new_row_df = pd.DataFrame.from_dict(new_row, orient='index').T

        dff = pd.concat([dff, new_row_df], ignore_index=True)
    except:
        continue

174 

/run/user/1000/app/org.jupyter.JupyterLab/ipykernel_444/264117204.py:39: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sections[3], 'html.parser')


954 

/run/user/1000/app/org.jupyter.JupyterLab/ipykernel_444/264117204.py:25: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sections[1], 'html.parser')


1465 

/run/user/1000/app/org.jupyter.JupyterLab/ipykernel_444/264117204.py:32: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sections[2], 'html.parser')


2515 

/run/user/1000/app/org.jupyter.JupyterLab/ipykernel_444/264117204.py:32: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sections[2], 'html.parser')


2841 

In [80]:
dff

,Вакансия,Обязанности,Требования,Условия
0,Fullstack-разработчик уровня middle +,Работа над корпоративным порталом (Битрикс24)...,Умение разбираться в чужом кодеОпыт разработки...,Работа удаленно\nРабота удаленно\n
1,Менеджер по продажам,"Достижение плановых показателей\nОрганизация, ...",От 3 лет в b2b продажах \n Опыт активных прод...,Оклад+ Процент от продаж\nОформление в соответ...
2,Помощник (стажер) программиста 1С,Консультирование пользователей по работе в про...,Знанием основных механизмов платформы 1С: каки...,График работы 5/2.\nЗарплата по результатам со...
3,Монтажник слаботочных систем,"Монтаж слаботочных систем: компьютерных сетей,...","Опыт работы от трех лет,\nОтветственность, исп...","65 000 р. на испытательный срок, далее по резу..."
4,Ассистент менеджера по продажам,Работа с существующей клиентской базой: консул...,"Коммуникабельность, грамотная речь, знание 1С....","Оформление по ТК РФ, график работы 5/2 с 10:00..."
...,...,...,...,...
2672,Сотрудник охраны в ночь,,сменный график работы: 3/3 по 12 часов в ночны...,обеспечение безопасности сотрудников и посети...
2673,Продавец/консультант,,сменный график работы: 3/3 по 11 часов;\nофици...,"работать на кассе, консультировать покупателей..."
2674,Менеджер по дистанционному обучению персонала,,"Разработка электронных курсов, дистанционного ...",Высшее образование;\nОпыт в разработке учебных...
2675,Руководитель отдела кадрового администрирования,Ведение кадрового делопроизводства в полном об...,Высшее образование;\nЗнание трудового законода...,Оформление согласно ТК РФ;\nГрафик работы - 5/...


In [81]:
df2 = dff.copy()
df2 = df2.replace(r'^\s*$', np.nan, regex=True)
df2 = df2.dropna()
df2.reset_index(inplace=True)

In [82]:
df2

,index,Вакансия,Обязанности,Требования,Условия
0,0,Fullstack-разработчик уровня middle +,Работа над корпоративным порталом (Битрикс24)...,Умение разбираться в чужом кодеОпыт разработки...,Работа удаленно\nРабота удаленно\n
1,1,Менеджер по продажам,"Достижение плановых показателей\nОрганизация, ...",От 3 лет в b2b продажах \n Опыт активных прод...,Оклад+ Процент от продаж\nОформление в соответ...
2,2,Помощник (стажер) программиста 1С,Консультирование пользователей по работе в про...,Знанием основных механизмов платформы 1С: каки...,График работы 5/2.\nЗарплата по результатам со...
3,3,Монтажник слаботочных систем,"Монтаж слаботочных систем: компьютерных сетей,...","Опыт работы от трех лет,\nОтветственность, исп...","65 000 р. на испытательный срок, далее по резу..."
4,4,Ассистент менеджера по продажам,Работа с существующей клиентской базой: консул...,"Коммуникабельность, грамотная речь, знание 1С....","Оформление по ТК РФ, график работы 5/2 с 10:00..."
...,...,...,...,...,...
1238,2666,Инженер по информационной безопасности,администрирование используемых средств защиты ...,"навыки администрирования Mikrotik, Cisco ASA, ...",стабильную и разноплановую работу в крупной ко...
1239,2667,Помощник мастера по обслуживанию супермаркета,стабильная выплата заработной платы 2 раза в м...,помощь мастеру в обеспечении бесперебойной раб...,образование: электро-техническое (желательно);...
1240,2669,Водитель-экспедитор,получение груза в соответствии с товаросопрово...,"Знание: правил приемки-сдачи груза, оформления...",адрес начала и окончания маршрута: Тургенева 1...
1241,2670,Программист PHP,Поддержка текущей кодовой базы и разработка но...,Опыт коммерческой разработки на PHP от 1 года;...,Стабильную и разноплановую работу в крупной ко...


In [83]:
df2.to_parquet('/home/marina/hh_ru/df.parquet.gzip', compression='gzip')